In [1]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('images/5725_HASAN_MARAE Optic nerve.png_ right.png')

# Define the color you want to remove (in BGR format)
color_to_remove = np.array([ 45 , 33 ,143])  # For example, remove red color (in BGR format)

# Define the tolerance for color matching (you can adjust this value)
tolerance = 50

# Create lower and upper bounds for color matching
lower_bound = color_to_remove - tolerance
upper_bound = color_to_remove + tolerance

# Mask the image to identify pixels within the specified color range
mask = cv2.inRange(image, lower_bound, upper_bound)

# Invert the mask to keep the pixels outside the specified color range
inverse_mask = cv2.bitwise_not(mask)

# Apply the mask to remove the specified color from the image
result = cv2.bitwise_and(image, image, mask=inverse_mask)

# Display the original and resulting images
cv2.imshow('Original Image', image)
cv2.imshow('Result Image', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('images/5725_HASAN_MARAE Optic nerve.png_ right.png')

# Define the color you want to remove (in BGR format)
color_to_remove = np.array([ 32 ,220 , 22])  # For example, remove red color (in BGR format)

# Define the tolerance for color matching (you can adjust this value)
tolerance = 50

# Create lower and upper bounds for color matching
lower_bound = color_to_remove - tolerance
upper_bound = color_to_remove + tolerance

# Iterate through the image and replace each removed pixel with its neighbor
for y in range(image.shape[0]):
    for x in range(image.shape[1]):
        # Check if the pixel color is within the specified color range
        if (lower_bound <= image[y, x]).all() and (image[y, x] <= upper_bound).all():
            # Replace the pixel color with the color of the neighboring pixel
            if y > 0 and x > 0:
                image[y, x] = image[y-1, x-1]
            elif y > 0:
                image[y, x] = image[y-1, x]
            elif x > 0:
                image[y, x] = image[y, x-1]

# Display the resulting image
cv2.imshow('Result Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [6]:
import cv2
import numpy as np

def select_roi(image):
    # Display the image and allow the user to select a ROI
    roi = cv2.selectROI("Select ROI", image, fromCenter=False, showCrosshair=False)
    cv2.destroyAllWindows()
    return roi

def get_dominant_color(image, roi):
    # Extract the selected ROI from the image
    x, y, w, h = roi
    selected_area = image[y:y+h, x:x+w]

    # Flatten the selected area to a list of pixels
    pixels = selected_area.reshape(-1, 3)

    # Calculate the histogram of RGB values
    hist = cv2.calcHist([pixels], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])

    # Find the index of the most dominant color
    b, g, r = np.unravel_index(hist.argmax(), hist.shape)

    return (r, g, b)

# Load the image
image = cv2.imread('images/5725_HASAN_MARAE Optic nerve.png_ right.png')

# Select the ROI
roi = select_roi(image)

# Get the dominant color within the selected area
dominant_color = get_dominant_color(image, roi)

# Display the dominant color
print("Dominant color (RGB):", dominant_color)

# Optionally, you can convert the RGB values to the corresponding color name

# Show the image with the selected ROI
cv2.rectangle(image, (roi[0], roi[1]), (roi[0] + roi[2], roi[1] + roi[3]), (0, 255, 0), 2)
cv2.imshow('Image with ROI', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import cv2

def get_pixel_color(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        pixel = image[y, x]  # Get the BGR values of the clicked pixel
        print("Selected pixel color (BGR):", pixel)

# Load the image
image = cv2.imread('images/5725_HASAN_MARAE Optic nerve.png_ right.png')

# Display the image and set up the mouse callback function
cv2.imshow('Select Pixel', image)
cv2.setMouseCallback('Select Pixel', get_pixel_color)

# Wait for the user to click on the image
cv2.waitKey(0)
cv2.destroyAllWindows()


Selected pixel color (BGR): [ 26 212  15]
Selected pixel color (BGR): [ 16 144   0]
Selected pixel color (BGR): [ 70 148   9]
Selected pixel color (BGR): [ 21 209  11]
Selected pixel color (BGR): [ 27 212  11]
Selected pixel color (BGR): [59 42  3]


In [1]:
import cv2
import numpy as np

def count_colors(image_path, k=8):
    # Read image
    image = cv2.imread(image_path)
    
    # Convert image to RGB (if not already)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Flatten the image to 1D array
    pixels = image_rgb.reshape(-1, 3)
    
    # Convert to float32 for k-means
    pixels = np.float32(pixels)
    
    # Apply k-means clustering
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Count unique colors
    unique_colors = np.unique(centers, axis=0)
    num_colors = len(unique_colors)
    
    return num_colors

# Example usage
image_path = "images/5725_HASAN_MARAE Optic nerve.png_ right.png"
num_colors = count_colors(image_path)
print("Number of colors in the image:", num_colors)


Number of colors in the image: 8


In [2]:
import cv2
import numpy as np

def count_colors(image_path, k=8):
    # Read image
    image = cv2.imread(image_path)
    
    # Convert image to RGB (if not already)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Flatten the image to 1D array
    pixels = image_rgb.reshape(-1, 3)
    
    # Convert to float32 for k-means
    pixels = np.float32(pixels)
    
    # Apply k-means clustering
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Count unique colors and their frequencies
    unique_colors, counts = np.unique(labels, return_counts=True)
    total_pixels = np.sum(counts)
    color_rates = counts / total_pixels
    
    return unique_colors, color_rates

# Example usage
image_path = "images/5725_HASAN_MARAE Optic nerve.png_ right.png"
unique_colors, color_rates = count_colors(image_path)
for color, rate in zip(unique_colors, color_rates):
    print("Color:", color, "Rate:", rate)


Color: 0 Rate: 0.028653617397669737
Color: 1 Rate: 0.020997275259765347
Color: 2 Rate: 0.0035774924256288254
Color: 3 Rate: 0.04525496146729295
Color: 4 Rate: 0.011285304703226987
Color: 5 Rate: 0.006429954858781187
Color: 6 Rate: 0.7913807214461457
Color: 7 Rate: 0.09242067244148926


In [1]:
colors = {}
data = []

In [5]:
import cv2
import numpy as np
import webcolors




def rgb_to_name(rgb_tuple):
    try:
        color_name = webcolors.rgb_to_name(rgb_tuple)
    except ValueError:
        # If the RGB value doesn't match any named color, return None
        color_name = None
    return color_name

def count_colors(image_path, k=4):
    # Read image
    image = cv2.imread(image_path)
    
    # Convert image to RGB (if not already)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Flatten the image to 1D array
    pixels = image_rgb.reshape(-1, 3)
    
    # Convert to float32 for k-means
    pixels = np.float32(pixels)
    
    # Apply k-means clustering
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Count unique colors and their frequencies
    unique_labels, counts = np.unique(labels, return_counts=True)
    total_pixels = np.sum(counts)
    color_rates = counts / total_pixels
    
    # Convert cluster centers to uint8 for color representation
    centers = np.uint8(centers)
    
    # Initialize lists to store color information
    colors = []
    color_names = []
    
    # Map RGB values to color names
    for color in centers:
        color_name = rgb_to_name(tuple(color))
        colors.append(color)
        color_names.append(color_name)
    
    return colors, color_names, color_rates

# Example usage
image_path = "images/5725_HASAN_MARAE Optic nerve.png_ right2.png"
colors, color_names, color_rates = count_colors(image_path)
for color, color_name, rate in zip(colors, color_names, color_rates):
    data.append([color,rate])
    print("Color:", color, "Rate:", rate)


# colors.append({'image':data})    


Color: [ 82 245  38] Rate: 0.050214957917970364
Color: [ 4 56 75] Rate: 0.7092368235950424
Color: [  5  99 131] Rate: 0.14566809769822212
Color: [ 22 174 203] Rate: 0.09488012078876519


In [19]:
colors

[array([ 92, 244,  33], dtype=uint8),
 array([ 22, 121, 158], dtype=uint8),
 array([ 8, 51, 67], dtype=uint8),
 array([ 61, 219, 126], dtype=uint8),
 array([ 37, 169, 205], dtype=uint8),
 array([211, 216, 221], dtype=uint8),
 array([ 17,  82, 105], dtype=uint8),
 array([125, 141, 142], dtype=uint8),
 {'image': [[array([ 61, 220, 124], dtype=uint8), 0.023749974582647066],
   [array([ 92, 245,  31], dtype=uint8), 0.025928241729193356],
   [array([ 84, 107, 106], dtype=uint8), 0.012174912055958845],
   [array([ 23, 125, 161], dtype=uint8), 0.06898523760141524],
   [array([189, 195, 201], dtype=uint8), 0.010680371703369324],
   [array([ 37, 171, 206], dtype=uint8), 0.07168201874783953],
   [array([ 14,  83, 108], dtype=uint8), 0.09203115150775737],
   [array([ 8, 51, 67], dtype=uint8), 0.6947680920718192],
   [array([ 92, 244,  33], dtype=uint8), 0.026718721405477947],
   [array([ 22, 121, 158], dtype=uint8), 0.06939445698367189],
   [array([ 8, 51, 67], dtype=uint8), 0.6910291994550519],


In [8]:
import cv2
import numpy as np

def identify_colors(image_path, num_clusters=5):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Reshape the image into a 2D array of pixels
    pixels = image_rgb.reshape(-1, 3)
    
    # Convert the pixels to float32 for K-means clustering
    pixels = np.float32(pixels)
    
    # Define criteria for K-means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    
    # Apply K-means clustering
    _, labels, centers = cv2.kmeans(pixels, num_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Convert the centers to uint8
    centers = np.uint8(centers)
    
    # Initialize a dictionary to store the counts of each color
    color_counts = {}
    
    # Loop through each pixel and count the occurrences of each color cluster
    for label in np.unique(labels):
        color = tuple(centers[label])
        count = np.sum(labels == label)
        color_counts[color] = count
    
    return color_counts

# Example usage
image_path = "images/5725_HASAN_MARAE Optic nerve.png_ right2.png"
color_counts = identify_colors(image_path)
print("Color Counts:", color_counts)


Color Counts: {(19, 165, 207): 58717, (4, 56, 74): 461547, (5, 96, 128): 99033, (90, 247, 19): 25374, (50, 232, 127): 14993}


In [1]:
import cv2
import numpy as np

def identify_colors(image_path, num_clusters=5):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Reshape the image into a 2D array of pixels
    pixels = image_rgb.reshape(-1, 3)
    
    # Convert the pixels to float32 for K-means clustering
    pixels = np.float32(pixels)
    
    # Define criteria for K-means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    
    # Apply K-means clustering
    _, labels, centers = cv2.kmeans(pixels, num_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Convert the centers to uint8
    centers = np.uint8(centers)
    
    # Initialize a dictionary to store the counts of each color
    color_counts = {}
    
    # Loop through each pixel and count the occurrences of each color cluster
    for label in np.unique(labels):
        color = tuple(centers[label])
        count = np.sum(labels == label)
        color_counts[color] = count
    
    # Visualize the image with color clusters
    cluster_labels = np.uint8(labels.reshape(image_rgb.shape[:2]))
    clustered_image = centers[cluster_labels]
    
    return color_counts, clustered_image

# Example usage
image_path = "images/5725_HASAN_MARAE Optic nerve.png_ left.png"
color_counts, clustered_image = identify_colors(image_path)

# Show the original and clustered images
cv2.imshow("Original Image", cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
cv2.imshow("Clustered Image", cv2.cvtColor(clustered_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Color Counts:", color_counts)


Color Counts: {(36, 165, 193): 43758, (183, 190, 196): 4633, (8, 52, 68): 284408, (84, 237, 58): 16136, (22, 98, 126): 44497}


In [1]:
import cv2
import numpy as np

def identify_colors(image_path, num_clusters=5, roi=None):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Define the ROI if provided
    if roi is not None:
        x, y, w, h = roi
        image_rgb = image_rgb[y:y+h, x:x+w]
    
    # Reshape the image into a 2D array of pixels
    pixels = image_rgb.reshape(-1, 3)
    
    # Convert the pixels to float32 for K-means clustering
    pixels = np.float32(pixels)
    
    # Define criteria for K-means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    
    # Apply K-means clustering
    _, labels, centers = cv2.kmeans(pixels, num_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Convert the centers to uint8
    centers = np.uint8(centers)
    
    # Initialize a dictionary to store the counts of each color
    color_counts = {}
    
    # Loop through each pixel and count the occurrences of each color cluster
    for label in np.unique(labels):
        color = tuple(centers[label])
        count = np.sum(labels == label)
        color_counts[color] = count
    
    # Visualize the image with color clusters
    cluster_labels = np.uint8(labels.reshape(image_rgb.shape[:2]))
    clustered_image = centers[cluster_labels]
    
    return color_counts, clustered_image

# Example usage
image_path = "images/5725_HASAN_MARAE Optic nerve.png_ left.png"
roi = (100, 100, 200, 200)  # Define the ROI as (x, y, width, height)
color_counts, clustered_image = identify_colors(image_path, roi=roi)

# Show the original and clustered images
cv2.imshow("Original Image", cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
cv2.imshow("Clustered Image", cv2.cvtColor(clustered_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Color Counts:", color_counts)


Color Counts: {(33, 159, 199): 9140, (17, 99, 129): 8271, (102, 245, 31): 2195, (9, 55, 72): 18774, (62, 213, 117): 1620}
